## Fitting Routine and Pickle Generation

Use a Gaussian fitting routine and powerlaw corrections to store important information about each spectra in a pickle (.pkl) file. Contents include:  

* 

In [1]:
from libraries import *
from functions import *
from fitting import *
from util import *


vims_wave = np.loadtxt('/home/alanyu/Dropbox (MIT)/VIMS_UROP/vims_wave.txt') 
data[131]

/home/alanyu/anaconda3/lib/python3.8/site-packages/pysis/env.py:33: RuntimeWarning: Could not find isis. Is `ISISROOT` set?
  warnings.warn('Could not find isis. Is `ISISROOT` set?', RuntimeWarning)


Current progress: 50000
Current progress: 100000
Current progress: 150000
Current progress: 200000
Current progress: 250000
DONE


{'name': 'V1649176775_1.ir_rc20final.cub',
 'sequenceid': 'S59',
 'observationid': 'VIMS_129TI_REGMAP001_ISS',
 'latitude': 0.030465888593957,
 'longitude': 111.27401231818,
 'phase': 53.461344024672,
 'incidence': 64.811856003527,
 'emission': 11.357761008738,
 'distance': 10446.338013075,
 'lineres': 4123.9872648578,
 'sampres': 3911.4676750721,
 'scposx': -2297.4339402712,
 'scposy': 10190.360488278,
 'scposz': 65.789759507016,
 'sunposx': 976824913.11096,
 'sunposy': 1030025422.6262,
 'sunposz': 93570399.422326,
 'ettime': 323755251.99934,
 'starttime': '2010-04-05 15:55:01.511',
 'exposureduration': 80.0,
 'spectrum': array([ 3.75123e-02,  5.80524e-02,  7.95100e-02,  8.85571e-02,
         6.69775e-02,  4.51409e-02,  3.68548e-02,  3.50120e-02,
         4.35997e-02,  5.58487e-02,  7.22301e-02,  8.14924e-02,
         7.40312e-02,  4.64825e-02,  2.56170e-02,  1.88336e-02,
         1.39813e-02,  1.41861e-02,  1.90291e-02,  2.72796e-02,
         3.59699e-02,  3.71576e-02,  4.59293e-02, 

In [4]:
%matplotlib qt

for i in range(100,200):
    if fit(data[i]['spectrum'], True) == {}:
        print(i)
    
def test_fit(i, channel = '2.0'):
    result = fit(data[i]['spectrum'], True)
    print(result)
    plt.plot(vims_wave,data[i]['spectrum'], label = 'original')
    
    if 'powerlaw' in result:
        plt.plot(vims_wave, data[i]['spectrum'] - powerlaw_no_constant(vims_wave, result['powerlaw']['scale'], result['powerlaw']['exponent']), label = 'Powerlaw Adjustment')
    
    plt.plot(vims_wave, gaussian(vims_wave, *result[channel]['gaussian'].values()), label = 'Gaussian')
    plt.plot(vims_wave, skew(vims_wave, *(result[channel]['skew'][x] for x in result[channel]['skew'] if 'peak' not in x)), label = 'Skew')
    plt.legend()
test_fit(13
         , '2.0')

100
104
119
124
133
137
154
164
{'1.2': {'gaussian': {'mean': 1.2707801211411223, 'sigma': -0.026745474356244997, 'amp': 0.08514593440482354}, 'skew': {'A': 0.005784864050315544, 'e': 1.2974522957279206, 'w': 0.04418363631482148, 'a': -3.5257581444671944, 'peak_channel': 1.27813, 'peak_value': 0.08909500508732324}, 'raw_peak_channel': 1.27813, 'raw_peak_value': 0.099412138334084}, '1.6': {'gaussian': {'mean': 1.5614566769317462, 'sigma': 0.05293952305352023, 'amp': 0.05325478905654311}, 'skew': {'A': 0.0071939767238004704, 'e': 1.6161792051156587, 'w': 0.08660421383648692, 'a': -2.9332039595552484, 'peak_channel': 1.57361, 'peak_value': 0.05434958161247187}, 'raw_peak_channel': 1.57361, 'raw_peak_value': 0.05985050474889782}, '2.0': {'gaussian': {'mean': 2.026625869629559, 'sigma': 0.05879105206589942, 'amp': 0.046457253239715834}, 'skew': {'A': 0.006979598967841126, 'e': 2.07548973490379, 'w': 0.08291989040436869, 'a': -1.5255195566857287, 'peak_channel': 2.03424, 'peak_value': 0.0460

### Define skewed gaussian

PDF:
$\phi(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{\frac{-x^2}{2}}$  

CDF:
$\Phi(x) = \frac{1}{2}\left[1+\textrm{erf}\left(\frac{x}{\sqrt{2}}\right)\right]$



In [10]:
%matplotlib qt
skewed_gaussian_initial_guesses = {'1.2': (1, 1.2, .1, .1), '1.6': (1, 1., .1, .1), '2.0': (1, 2.0, .1, .1), '1.0': (1, 1.0, .1, .1)}
fit_test(213, data, skew, skewed_gaussian_initial_guesses, window = '1.0')

Max channel: 1.06541
Powerlaw: [ 0.02319542 -2.02316714] 
 Gaussian: [1.059658   0.04866083 0.05609375] 
 Powerlaw Gaussian: [1.06319388 0.02943952 0.03940249] 
 Function: [ 0.13668238  0.73016903  0.1885804  -5.77906197] 
 Powerlaw Function [ 0.00299596  1.09434944  0.04889036 -2.76068798]


In [19]:
%matplotlib qt

def view_stefan_correction(i):
    spectra = data[i]['spectrum']

    plt.plot(spectra, label = 'Original')

    plt.plot(spectra/f(np.radians(data[i]['incidence']),np.radians(data[i]['emission']),np.radians(data[i]['phase'])), label = "Stefan Correction")

    plt.legend()
    
view_stefan_correction(4221)

In [21]:
skew

<function functions.skew(x, A=0.15, e=0, w=1, a=0)>

In [28]:
file = 'belet_V1563500997_1_003.ir_rc20final_0.95637911406159_92.900238213905'

In [29]:
with open(f"new_pixels/belet/{file}.pkl", 'rb') as f:
    x = pickle.load(f)

In [30]:
x

{'name': 'V1563500997_1_003.ir_rc20final.cub',
 'sequenceid': 'S32',
 'observationid': 'VIMS_048TI_VHIRESNAC001_PRIME',
 'latitude': 0.95637911406159,
 'longitude': 92.900238213905,
 'phase': 17.220176189372,
 'incidence': 15.708082441468,
 'emission': 1.9256560207006,
 'distance': 4183.5869046142,
 'lineres': 620.56454535385,
 'sampres': 769.28732395324,
 'scposx': -264.52310884008,
 'scposy': 4174.4361839831,
 'scposz': 80.681219635289,
 'sunposx': 162746463.23403,
 'sunposy': 1343305189.5868,
 'sunposz': -271874927.96216,
 'ettime': 238079793.04861,
 'starttime': '2007-07-19 01:15:28.292',
 'exposureduration': 13.0,
 'spectrum': array([ 5.07852e-02,  8.84350e-02,  1.28726e-01,  1.75767e-01,
         1.26202e-01,  7.73446e-02,  5.86264e-02,  5.39683e-02,
         5.84070e-02,  7.97667e-02,  1.20212e-01,  1.56342e-01,
         1.51922e-01,  8.43662e-02,  4.83023e-02,  3.09434e-02,
         2.43068e-02,  2.22335e-02,  2.66279e-02,  3.39844e-02,
         4.86339e-02,  5.24678e-02,  7.63

In [31]:
def plot_result(entry):
    plt.plot(vims_wave,entry['spectrum'], label = 'Original')
    plt.plot(vims_wave,entry['stefan_spectrum'], label = 'Stefan')

In [38]:
plot_result(x)

In [33]:
s = [0.00432699, 0.0112928, 0.0133825, 0.0189596, 0.0140335, 0.00832937, 0.00520691, 0.00718557, 0.00473012, 0.0119746, 0.0121561, 0.0175457, 0.0215297, 0.0118668, 0.00996025, 0.0052169, 0.00500274, 0.00514135, 6.21552e-16, 0.00285062, 0.00280052, 0.00851847, 0.00292388, 0.00908544, 0.018682, 0.00963079, 0.00344781, -0.00360104, 9.76767e-16, 1.04769e-15, 0.00385669, 1.42116e-15, 1.00284e-15, 0.00454352, 0.00451307, 0.00904659, 1.2567e-15, 0.00497627, 0.00516394, 1.65497e-15, 0.0055415, 0.00583813, 0.0121814, 0.00617359, 0.00673096, 0.0161536, 7.62451e-15, 0.0104326, 0.00771741, 2.24059e-15, -0.00785823, 0.00752564, -0.0074543, 2.53019e-15, 1.53716e-15, 2.25995e-15, -0.00795433, -0.00801811, 1.87864e-15, 2.7908e-15, 2.63172e-15, 2.78682e-15, 0.0076589, 1.74588e-15, 0.0163735, 2.46842e-15, -0.00894701, -0.00867962, 0.0161689, 0.0233235, 0.00788433, 0.00821806, 0.00858231, 0.00849267, 0.0164613, 0.00798333, 0.00784867, 1.84433e-15, 2.08885e-15, 0.00852424, 2.52364e-15, 2.0746e-15, 0.0078044, 1.74403e-15, 1.97881e-15, 1.81416e-15, -0.00825704, 2.13976e-15, 0.00832081, 0.00809493, 2.22815e-15, 1.76187e-15, 2.18368e-15, 0.00788844, 2.0661e-15, 0.00829416, -0.00838485, -0.00845728, 2.26257e-15, 2.03197e-15, 0.00847458, 2.40354e-15, -0.00829408, 0.016215, -0.00793823, 0.0155159, 1.52935e-15, 1.51722e-15, 2.21685e-15, 0.0158852, 0.0162315, 2.19768e-15, 0.00815162, 2.05014e-15, 0.00784858, 0.0230774, 0.00737698, 0.0147817, 0.00749921, 0.00763562, 0.00781847, 2.25288e-15, 0.00804673, 0.00819195, 1.77706e-15, -0.00848073, 2.78378e-15, 3.91738e-15, 0.0114119, 0.00904034, 0.00925854, 0.00933178, 1.8877e-15, 1.67004e-15, 2.02149e-15, 1.77478e-15, -0.0104454, 2.04308e-15, 0.0100468, -0.00993237, 0.00999634, 1.58611e-15, -0.0103915, 1.80835e-15, -0.0108116, 1.85623e-15, 2.66554e-15, 1.70765e-15, 0.0107249, -0.010526, 2.08039e-15, 1.77755e-15, -0.0106011, 2.13263e-15, 2.46772e-15, 2.16754e-15, -0.0110157, 2.20982e-15, 0.0113302, 0.0115923, 0.0119426, 3.31249e-15, -0.0125242, -0.0127515, 3.65676e-15, 0.0130484, -0.0134264, 3.73483e-15, 2.85721e-15, 3.58795e-15, 0.0148688, 0.0149866, -0.01488, -0.0149638, 2.98532e-15, -0.0152672, 5.24559e-15, 0.0310176, -0.0156586, 7.24261e-15, 2.80451e-14, -0.0297617, -0.0185096, 0.018483, 0.0188252, 3.77922e-15, 4.41656e-15, 4.22121e-15, 6.25708e-15, 3.99676e-15, 4.60889e-15, 4.99875e-15, -0.0417119, -0.0419227, 6.0459e-15, 0.0214183, 6.80068e-15, 0.0220222, -0.0223868, 0.0226201, 7.24096e-15, 4.99282e-15, -0.0237357, 6.18209e-15, -0.0246987, -0.0251998, 0.0261825, 7.10393e-15, 0.0270804, -0.0280683, -0.0284423, 4.95274e-15, -0.0301292, 7.00855e-15, 0.0310578, 4.4947e-15, 0.0325566, 5.56802e-15, 0.0332294, 5.82197e-15, 1.19684e-14, 0.03566, -0.0367493, 6.42974e-15, 0.0386296, 0.0396051, 1.37614e-14, -0.0413042, 1.17333e-14, 7.09821e-15, -0.0420182, -0.0422562, 1.32421e-14, 6.64802e-15, 2.70611e-14, 6.11786e-15, -0.0427865, -0.0438324, -0.0448996, 0.0447978, -0.0458629, 5.35996e-15, 1.09649e-14, 0.0979968, 0.0502889, 5.5251e-15, 0.0534105, 0.0555425, 1.38736e-14, 6.31794e-15, 1.50302e-14, 7.44019e-15, 1.38821e-14, 8.4602e-15, 0.0808798, 0.173793]


In [35]:
fit(s, True)

{}

In [37]:
plt.plot(vims_wave, s)